<a href="https://colab.research.google.com/github/CodeHunterOfficial/ABC_DataMining/blob/main/NLP/NLP-2025/WordPiece_%D0%A2%D0%BE%D0%BA%D0%B5%D0%BD%D0%B8%D0%B7%D0%B0%D1%82%D0%BE%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**WordPiece Токенизатор**  
WordPiece Токенизатор представляет собой элегантный гибридный подход, разработанный для нахождения оптимального баланса между токенизацией на уровне слов и символов. Он разбивает слова на более мелкие, но семантически осмысленные единицы, называемые подсловами (subwords). Например, слово "неизвестные" может быть токенизировано как "не", "##извест", "##ные". Префикс ## указывает, что данный подтокен является продолжением предыдущего токена, а не началом нового слова.  
Основная идея WordPiece: Построить словарь токенов (включающий символы, подслова и, при необходимости, целые слова) таким образом, чтобы он был достаточно компактным, но при этом позволял эффективно представлять как известные, так и неизвестные слова, сохраняя при этом их лингвистический смысл.

**1. Принцип Работы WordPiece: Пошаговое Объяснение**  
Алгоритм WordPiece, подобно BPE, является итеративным методом, но отличается критерием выбора пар для слияния. Вместо простой частоты встречаемости, WordPiece использует метрику, основанную на вероятности или правдоподобии.  
Возьмем наш пример текста для обучения токенизатора: "В училище учитель учит ученикам по новому учебнику."

**1.1. Шаг 1: Предварительная Обработка Текста и Создание Начального Словаря**  
Перед началом обучения текст подвергается предварительной обработке:  
 Приведение к нижнему регистру: Все буквы переводятся в строчные, чтобы "Учитель" и "учитель" считались одним и тем же словом.  
 Удаление знаков препинания: Знаки препинания (кроме тех, что являются частью слова, например, в сокращениях, но в нашем примере их нет) удаляются или отделяются.  
 Разделение на слова: Текст сначала разбивается на слова по пробелам.  
Обработанный текст: "в училище учитель учит ученикам по новому учебнику"  
Начальный словарь (V₀): Начинаем с самого простого: каждый уникальный символ в нашем обработанном тексте становится отдельным токеном в нашем словаре. Пробел ( ) также является важным токеном, так как он помогает модели понимать границы слов.  
Уникальные символы в тексте: 'в', ' ', 'у', 'ч', 'и', 'л', 'щ', 'е', 'т', 'р', 'ь', 'н', 'к', 'а', 'м', 'п', 'о', 'б', 'ю'.  
V₀ = {'в', ' ', 'у', 'ч', 'и', 'л', 'щ', 'е', 'т', 'р', 'ь', 'н', 'к', 'а', 'м', 'п', 'о', 'б', 'ю'} (если бы были другие символы).  
Теперь мы можем представить весь наш обработанный текст как последовательность этих начальных токенов. Например, слово "училище" будет представлено как ['у', 'ч', 'и', 'л', 'и', 'щ', 'е'].  
Полная последовательность токенов в корпусе (после предварительной обработки): ['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'л', 'ь', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'к', 'а', 'м', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']  
Общее количество токенов в корпусе: 50.

**1.2. Шаг 2: Итеративное Слияние (Обучение Словаря)**  
Это ядро алгоритма WordPiece. Мы будем итеративно (повторяющимися шагами) объединять пары соседних токенов (биграммы), чтобы создавать новые, более длинные токены. Цель — найти такие пары, которые, будучи объединенными, максимально увеличивают "вероятность" (или "правдоподобие") нашего обучающего текста.  

**Формула оценки слияния (Score):** Для каждой пары соседних токенов A и B (биграммы AB) мы вычисляем "оценку слияния" по следующей формуле:  
$$
\text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$  
Где:  
• $f(A, B)$ — это количество раз, когда биграмма AB (токен A, за которым сразу следует токен B) встречается в текущем токенизированном корпусе.  
• $f(A)$ — это количество раз, когда токен A встречается в текущем токенизированном корпусе.  
• $f(B)$ — это количество раз, когда токен B встречается в текущем токенизированном корпусе.  

Данная формула измеряет, насколько часто A и B встречаются **вместе**, по сравнению с тем, насколько часто они встречаются **по отдельности**. Высокий балл означает, что A и B **очень часто появляются вместе**, и их слияние в один токен будет "полезным" для модели, так как оно позволяет более компактно (с меньшим количеством токенов) представить текст. Это эквивалентно максимизации правдоподобия обучающих данных, если мы рассматриваем процесс токенизации как генерацию последовательности токенов.




**Пример итераций с нашим текстом:**

**Итерация 1: Поиск первой лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'л', 'ь', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'к', 'а', 'м', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']`

- **Подсчет частот отдельных токенов (униграмм) в текущем корпусе:**
  - $f(\text{' '}) = 8$
  - $f(\text{'у'}) = 5$
  - $f(\text{'ч'}) = 5$
  - $f(\text{'и'}) = 5$
  - $f(\text{'л'}) = 2$
  - $f(\text{'щ'}) = 1$
  - $f(\text{'е'}) = 4$
  - $f(\text{'т'}) = 2$
  - $f(\text{'ь'}) = 1$
  - $f(\text{'н'}) = 2$
  - $f(\text{'к'}) = 2$
  - $f(\text{'а'}) = 1$
  - $f(\text{'м'}) = 2$
  - $f(\text{'п'}) = 1$
  - $f(\text{'о'}) = 3$
  - $f(\text{'в'}) = 2$ (один раз в начале "в", один — в "новому")
  - $f(\text{'б'}) = 1$


- **Подсчет частот биграмм (пар соседних токенов):**
  - $f(\text{'у', 'ч'}) = 5$ — в "училище", "учитель", "учит" (дважды), "ученикам", "учебнику"
  - $f(\text{'ч', 'и'}) = 5$ — аналогично
  - $f(\text{'и', 'л'}) = 1$ — в "училище"
  - $f(\text{'л', 'и'}) = 1$ — в "училище"
  - $f(\text{'и', 'щ'}) = 1$ — в "училище"
  - $f(\text{'щ', 'е'}) = 1$ — в "училище"
  - $f(\text{'е', ' '}) = 2$ — после "училище", "учитель"
  - $f(\text{'и', 'т'}) = 2$ — в "учит" (дважды)
  - $f(\text{'т', 'е'}) = 1$ — в "учитель"
  - $f(\text{'е', 'л'}) = 1$ — в "учитель"
  - $f(\text{'л', 'ь'}) = 1$ — в "учитель"
  - $f(\text{'ч', 'е'}) = 2$ — в "ученикам", "учебнику"
  - $f(\text{'е', 'н'}) = 1$ — в "ученикам"
  - $f(\text{'н', 'и'}) = 2$ — в "ученикам", "учебнику"
  - $f(\text{'и', 'к'}) = 2$ — в "ученикам", "учебнику"
  - $f(\text{'к', 'а'}) = 1$ — в "ученикам"
  - $f(\text{'а', 'м'}) = 1$ — в "ученикам"
  - $f(\text{' ', 'п'}) = 1$ — перед "по"
  - $f(\text{'п', 'о'}) = 1$ — в "по"
  - $f(\text{'о', ' '}) = 1$ — после "по"
  - $f(\text{'н', 'о'}) = 2$ — в "новому", "учебнику"
  - $f(\text{'о', 'в'}) = 1$ — в "новому"
  - $f(\text{'в', 'о'}) = 1$ — в "новому"
  - $f(\text{'о', 'м'}) = 1$ — в "новому"
  - $f(\text{'м', 'у'}) = 1$ — в "новому"
  - $f(\text{'у', ' '}) = 4$ — после "в", "училище", "учитель", "новому"
  - $f(\text{'е', 'б'}) = 1$ — в "учебнику"
  - $f(\text{'б', 'н'}) = 1$ — в "учебнику"
  - $f(\text{'н', 'и'}) = 2$ — уже учтено
  - $f(\text{'и', 'к'}) = 2$ — уже учтено
  - $f(\text{'к', 'у'}) = 1$ — в "учебнику"
  - $f(\text{'в', ' '}) = 1$ — после слова "в"


- **Вычисление оценок слияния по формуле:**  
$$
  \text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$

  Примеры:
  - $\text{Score}(\text{'у', 'ч'}) = \frac{5}{5 \times 5} = \frac{5}{25} = 0.2$
  - $\text{Score}(\text{'ч', 'и'}) = \frac{5}{5 \times 5} = 0.2$
  - $\text{Score}(\text{'л', 'ь'}) = \frac{1}{2 \times 1} = \frac{1}{2} = 0.5$
  - $\text{Score}(\text{'к', 'а'}) = \frac{1}{2 \times 1} = 0.5$
  - $\text{Score}(\text{'а', 'м'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'б', 'н'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'п', 'о'}) = \frac{1}{1 \times 3} \approx 0.333$
  - $\text{Score}(\text{'н', 'о'}) = \frac{2}{2 \times 3} \approx 0.333$
  - $\text{Score}(\text{'щ', 'е'}) = \frac{1}{1 \times 4} = 0.25$
  - $\text{Score}(\text{'е', 'б'}) = \frac{1}{4 \times 1} = 0.25$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $\text{Score} = 0.5$ имеют биграммы: `'л', 'ь'`, `'к', 'а'`, `'а', 'м'`, `'б', 'н'`.  
  При равенстве оценок алгоритм может выбрать первую по порядку в лексикографическом или в порядке появления.  
  Предположим, выбирается первая появляющаяся такая пара — `'л', 'ь'` (в слове "учитель").

- **Слияние `'л', 'ь'`:**  
  Все вхождения последовательности `['л', 'ь']` заменяются на новый токен `'ль'`.  
  Например, слово "учитель" из `['у', 'ч', 'и', 'т', 'е', 'л', 'ь']` становится `['у', 'ч', 'и', 'т', 'е', 'ль']`.

- **Обновление словаря:**  
  Добавляем новый токен `'ль'` в словарь:  
$$
  V_1 = V_0 \cup \{\text{'ль'}\}
$$


**Итерация 2: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'к', 'а', 'м', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']`  
  (Токен `'ль'` уже заменил последовательность `'л', 'ь'` в слове "учитель".)  
  **Общее количество токенов: 49.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'л'}) = 1$ — всё ещё встречается в "училище" (`'и', 'л', 'и'`)
  - $f(\text{'ь'}) = 0$ — больше не встречается как отдельный токен
  - $f(\text{'ль'}) = 1$ — новый токен
  - Остальные частоты остаются без изменений:
    - $f(\text{'у'}) = 5$, $f(\text{'ч'}) = 5$, $f(\text{'и'}) = 5$, $f(\text{'е'}) = 4$, $f(\text{'т'}) = 2$, $f(\text{'к'}) = 2$, $f(\text{'а'}) = 1$, $f(\text{'м'}) = 2$, $f(\text{' '}) = 8$ и т.д.

- **Подсчет частот биграмм (обновленные):**
  - Биграмма `'л', 'ь'` больше не встречается.
  - Появилась новая биграмма: `'е', 'ль'` (из "учитель") → $f(\text{'е', 'ль'}) = 1$
  - Остальные биграммы, не затронутые заменой, сохраняют свои частоты:
    - $f(\text{'у', 'ч'}) = 5$, $f(\text{'ч', 'и'}) = 5$, $f(\text{'к', 'а'}) = 1$, $f(\text{'а', 'м'}) = 1$, $f(\text{'б', 'н'}) = 1$ и т.д.

- **Вычисление оценок слияния:**
  Используем формулу:
$$
  \text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$

  Примеры:
  - $\text{Score}(\text{'е', 'ль'}) = \frac{1}{4 \times 1} = 0.25$
  - $\text{Score}(\text{'к', 'а'}) = \frac{1}{2 \times 1} = 0.5$
  - $\text{Score}(\text{'а', 'м'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'б', 'н'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'у', 'ч'}) = \frac{5}{5 \times 5} = 0.2$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $\text{Score} = 0.5$ имеют биграммы: `'к', 'а'`, `'а', 'м'`, `'б', 'н'`.  
  Выбираем первую по порядку появления — `'к', 'а'` (в слове "ученикам").

- **Слияние `'к', 'а'`:**  
  Все вхождения последовательности `['к', 'а']` заменяются на новый токен `'ка'`.  
  Например, слово "ученикам" из `['у', 'ч', 'е', 'н', 'и', 'к', 'а', 'м']` становится `['у', 'ч', 'е', 'н', 'и', 'ка', 'м']`.

- **Обновление словаря:**  
  Добавляем новый токен `'ка'` в словарь:  
$$
  V_2 = V_1 \cup \{\text{'ка'}\}
$$



**Итерация 3: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'ка', 'м', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']`  
  (Токены `'ль'` и `'ка'` уже вставлены.)  
  **Общее количество токенов: 48.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'к'}) = 1$ — встречается в "учебнику"
  - $f(\text{'а'}) = 0$ — больше не встречается как отдельный токен
  - $f(\text{'ка'}) = 1$
  - $f(\text{'ам'}) = 1$ — пока нет, но биграмма `'а', 'м'` всё ещё может существовать
  - $f(\text{'а', 'м'}) = 1$ — в последовательности `['ка', 'м']` → `'ка', 'м'`, а не `'а', 'м'`  
    → Значит, биграмма `'а', 'м'` **больше не существует**, так как `'а'` исчез как отдельный токен

  Однако в слове "ученикам" теперь: `['ка', 'м']`, значит:
  - Биграмма: `'ка', 'м'`, а не `'а', 'м'`

  Следовательно:
  - $f(\text{'а', 'м'}) = 0$ → биграмма исчезла
  - $f(\text{'ка', 'м'}) = 1$ → новая биграмма

- **Подсчет частот биграмм:**
  - Биграммы `'к', 'а'` и `'а', 'м'` больше не встречаются.
  - Появилась новая биграмма: `'ка', 'м'` → $f = 1$
  - Остальные биграммы без изменений, например:
    - $f(\text{'б', 'н'}) = 1$
    - $f(\text{'у', 'ч'}) = 5$
    - $f(\text{'ч', 'и'}) = 5$

- **Вычисление оценок:**
  - $\text{Score}(\text{'ка', 'м'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'б', 'н'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'у', 'ч'}) = \frac{5}{5 \times 5} = 0.2$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $0.5$ имеют `'ка', 'м'` и `'б', 'н'`.  
  Выбираем `'ка', 'м'` как следующую по порядку.

- **Слияние `'ка', 'м'`:**  
  Заменяем последовательность `['ка', 'м']` на новый токен `'кам'`.  
  Слово "ученикам" из `['у', 'ч', 'е', 'н', 'и', 'ка', 'м']` становится `['у', 'ч', 'е', 'н', 'и', 'кам']`.

- **Обновление словаря:**  
  Добавляем новый токен `'кам'` в словарь:  
$$
  V_3 = V_2 \cup \{\text{'кам'}\}
$$



**Итерация 4: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']`  
  (Токены `'ль'`, `'ка'`, `'ам'` уже объединены в `'кам'`. Уточнение: `'ам'` как отдельный токен не был добавлен — слияние произошло напрямую в `'кам'`.)  
  **Общее количество токенов: 47.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'а'}) = 0$ — больше не встречается
  - $f(\text{'м'}) = 1$ — встречается в "новому" (`'м', 'у'`)
  - $f(\text{'кам'}) = 1$
  - Остальные частоты без изменений.

- **Подсчет частот биграмм (обновленные):**
  - Биграмма `'и', 'кам'` появляется один раз (в "ученикам").
  - $f(\text{'и', 'кам'}) = 1$
  - Остальные биграммы, не затронутые слиянием, сохраняют свои значения:
    - $f(\text{'б', 'н'}) = 1$ — в "учебнику"
    - $f(\text{'у', 'ч'}) = 5$
    - $f(\text{'ч', 'и'}) = 5$

- **Вычисление оценок слияния:**
$$
  \text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$
  - $\text{Score}(\text{'и', 'кам'}) = \frac{1}{5 \times 1} = 0.2$
  - $\text{Score}(\text{'б', 'н'}) = \frac{1}{1 \times 2} = 0.5$
  - $\text{Score}(\text{'у', 'ч'}) = \frac{5}{5 \times 5} = 0.2$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $0.5$ имеет биграмма `'б', 'н'`.

- **Слияние `'б', 'н'`:**  
  Заменяем последовательность `['б', 'н']` на новый токен `'бн'`.  
  Слово "учебнику" из `['у', 'ч', 'е', 'б', 'н', 'и', 'к', 'у']` становится `['у', 'ч', 'е', 'бн', 'и', 'к', 'у']`.

- **Обновление словаря:**  
  Добавляем новый токен `'бн'` в словарь:  
$$
  V_4 = V_3 \cup \{\text{'бн'}\}
$$



**Итерация 5: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'н', 'о', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  (Токены `'ль'`, `'кам'`, `'бн'` добавлены.)  
  **Общее количество токенов: 46.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'б'}) = 0$
  - $f(\text{'н'}) = 2$ — встречается в "ученикам" (`'е', 'н'`) и "новому" (`'н', 'о'`)
  - $f(\text{'бн'}) = 1$


- **Подсчет частот биграмм:**
  - Появились новые биграммы:
    - $f(\text{'е', 'бн'}) = 1$ — в "учебнику"
    - $f(\text{'бн', 'и'}) = 1$ — в "учебнику"
  - $f(\text{'н', 'о'}) = 2$ — в "новому" и "учебнику"
  - $f(\text{'о', 'в'}) = 1$, $f(\text{'в', 'о'}) = 1$, $f(\text{'о', 'м'}) = 1$

- **Вычисление оценок:**
  - $\text{Score}(\text{'е', 'бн'}) = \frac{1}{4 \times 1} = 0.25$
  - $\text{Score}(\text{'бн', 'и'}) = \frac{1}{1 \times 5} = 0.2$
  - $\text{Score}(\text{'н', 'о'}) = \frac{2}{2 \times 3} \approx 0.333$
  - $\text{Score}(\text{'о', 'в'}) = \frac{1}{3 \times 2} \approx 0.167$
  - $\text{Score}(\text{'в', 'о'}) = \frac{1}{2 \times 3} \approx 0.167$

- **Выбор лучшей биграммы:**  
  Наивысшую оценку $\approx 0.333$ имеет `'н', 'о'`.

- **Слияние `'н', 'о'`:**  
  Заменяем все вхождения `['н', 'о']` на новый токен `'но'`.  
  Слово "новому" из `['н', 'о', 'в', 'о', 'м', 'у']` становится `['но', 'в', 'о', 'м', 'у']`.

- **Обновление словаря:**  
$$
  V_5 = V_4 \cup \{\text{'но'}\}
$$


**Итерация 6: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'но', 'в', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  **Общее количество токенов: 45.**

- **Подсчет частот:**
  - $f(\text{'но'}) = 1$
  - $f(\text{'в'}) = 2$ — в "в" и "новому"
  - $f(\text{'о'}) = 2$ — в "по", "новому"

- **Частоты биграмм:**
  - $f(\text{'но', 'в'}) = 1$ — в "новому"

- **Оценка слияния:**
$$
  \text{Score}(\text{'но', 'в'}) = \frac{1}{1 \times 2} = 0.5
$$

- **Выбор лучшей биграммы:**  
  Среди текущих биграмм `'но', 'в'` имеет одну из высоких оценок.

- **Слияние `'но', 'в'`:**  
  Заменяем `['но', 'в']` на новый токен `'нов'`.  
  Слово "новому" становится `['нов', 'о', 'м', 'у']`.

- **Обновление словаря:**  
$$
  V_6 = V_5 \cup \{\text{'нов'}\}
$$


**Итерация 7: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'нов', 'о', 'м', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  **Общее количество токенов: 44.**

- **Частоты:**
  - $f(\text{'нов'}) = 1$
  - $f(\text{'о'}) = 2$

- **Биграмма:**
  - $f(\text{'нов', 'о'}) = 1$

- **Оценка слияния:**
$$
  \text{Score}(\text{'нов', 'о'}) = \frac{1}{1 \times 2} = 0.5
$$

- **Слияние `'нов', 'о'`:**  
  Заменяем `['нов', 'о']` на `'ново'`.  
  Слово "новому" становится `['ново', 'м', 'у']`.

- **Обновление словаря:**  
$$
  V_7 = V_6 \cup \{\text{'ново'}\}
$$




**Итерация 8: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'ново', 'м', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  (Токены `'ль'`, `'кам'`, `'бн'`, `'но'`, `'нов'`, `'ново'` уже добавлены.)  
  **Общее количество токенов: 43.**

- **Подсчет частот отдельных токенов (обновленные):**
  - $f(\text{'ново'}) = 1$
  - $f(\text{'м'}) = 1$ — встречается в "новому"
  - $f(\text{'нов'}) = 0$ — больше не встречается как отдельный токен
  - Остальные частоты без изменений.

- **Подсчет частот биграмм:**
  - $f(\text{'ново', 'м'}) = 1$ — в слове "новому"

- **Оценка слияния:**
$$
  \text{Score}(\text{'ново', 'м'}) = \frac{f(\text{'ново', 'м'})}{f(\text{'ново'}) \times f(\text{'м'})} = \frac{1}{1 \times 1} = 1.0
$$

- **Выбор лучшей биграммы:**  
  Биграмма `'ново', 'м'` имеет высокую оценку (1.0).

- **Слияние `'ново', 'м'`:**  
  Заменяем последовательность `['ново', 'м']` на новый токен `'новом'`.  
  Слово "новому" из `['ново', 'м', 'у']` становится `['новом', 'у']`.

- **Обновление словаря:**  
  Добавляем новый токен `'новом'` в словарь:  
$$
  V_8 = V_7 \cup \{\text{'новом'}\}
$$



**Итерация 9: Поиск следующей лучшей биграммы**

- **Текущий корпус:**  
  `['в', ' ', 'у', 'ч', 'и', 'л', 'и', 'щ', 'е', ' ', 'у', 'ч', 'и', 'т', 'е', 'ль', ' ', 'у', 'ч', 'и', 'т', ' ', 'у', 'ч', 'е', 'н', 'и', 'кам', ' ', 'п', 'о', ' ', 'новом', 'у', ' ', 'у', 'ч', 'е', 'бн', 'и', 'к', 'у']`  
  (Токены `'новом'` и другие уже включены.)  
  **Общее количество токенов: 42.**

- **Подсчет частот отдельных токенов:**
  - $f(\text{'новом'}) = 1$
  - $f(\text{'у'}) = 5$ — встречается в "училище", "учитель", "учит", "ученикам", "учебнику"

- **Частота биграммы:**
  - $f(\text{'новом', 'у'}) = 1$

- **Оценка слияния:**
$$
  \text{Score}(\text{'новом', 'у'}) = \frac{1}{1 \times 5} = 0.2
$$



- **Выбор лучшей биграммы:**  
  На данном шаге среди всех доступных биграмм `'новом', 'у'` может быть одной из лучших, особенно если других высокочастотных пар не осталось.

- **Слияние `'новом', 'у'`:**  
  Заменяем `['новом', 'у']` на новый токен `'новому'`.  
  Слово "новому" теперь представлено как `['новому']`.

- **Обновление словаря:**  
  Добавляем новый токен `'новому'` в словарь:  
$$
  V_9 = V_8 \cup \{\text{'новому'}\}
$$



Мы продемонстрировали, как слово *"новому"* постепенно собиралось из отдельных символов через последовательные слияния:  
`'н'+'о'→'но'`, `'но'+'в'→'нов'`, `'нов'+'о'→'ново'`, `'ново'+'м'→'новом'`, `'новом'+'у'→'новому'`.  
Таким образом, WordPiece может формировать **целые слова** как отдельные токены, если они часто встречаются и слияние оправдано с точки зрения правдоподобия.

Этот процесс продолжается итеративно до достижения желаемого размера словаря. В реальных моделях, таких как BERT, он проходит **тысячи или десятки тысяч итераций** на огромных текстовых корпусах.

В конечном итоге, словарь содержит смесь:
- отдельных символов,
- подслов (например, `'уч'`, `'кам'`),
- целых слов (например, `'новому'`),  
что позволяет эффективно кодировать как частотные, так и редкие или неизвестные слова.



**2.3. Когда остановить итерацию? (Критерии остановки)**

Процесс итеративного слияния в WordPiece обычно останавливается при выполнении одного из следующих условий:

- **Достигнут заданный размер словаря:**  
  Это — наиболее распространённый критерий. Разработчик заранее определяет максимальное количество токенов в словаре (например, 30 000 для BERT, 50 000 для RoBERTa). Обучение завершается, как только словарь достигает этого предела.

- **Невозможно выполнить дальнейшие слияния:**  
  Это происходит, когда в корпусе больше нет соседних токенов, которые можно объединить — например, весь текст представлен минимальным числом токенов, или все возможные биграммы уже слиты.

- **Оценка лучшей биграммы падает ниже порога:**  
  Можно установить минимальное значение \(\text{Score}(A, B)\), ниже которого слияния считаются незначимыми. Если даже лучшая доступная биграмма имеет слишком низкую оценку, процесс останавливается, так как дальнейшие слияния вносят незначительный вклад в правдоподобие текста.




**Пример конечного словаря (Vfinal) после множества итераций:**

На основе последовательных итераций, наш словарь постепенно будет включать в себя следующие токены:  
`{'в', ' ', 'у', 'ч', 'и', 'л', 'щ', 'е', 'т', 'р', 'ь', 'н', 'к', 'а', 'м', 'п', 'о', 'б', 'ль', 'ка', 'ам', 'бн', 'но', 'нов', 'ново', 'новом', 'новому', 'уч', 'чи', 'ит', 'ищ', 'ще', 'е ', 'ни', 'ку', 'учи', 'учил', 'учили', 'училище', 'учитель', 'учит', 'уче', 'учени', 'ученик', 'ученикам', 'учеб', 'учебни', 'учебник', 'учебнику'}`.

- **Начальные символы:** Все уникальные символы из текста (например, `'в'`, `' '` , `'у'`, `'ч'` и т.д.) остаются в словаре — они необходимы для токенизации редких или неизвестных слов.
- **Итеративные слияния:** В процессе обучения добавляются составные токены: `'ль'`, `'ка'`, `'бн'`, `'но'`, `'нов'`, `'ново'`, `'новом'`, `'новому'` — как результат последовательных оптимальных слияний.
- **Более длинные подслова и целые слова:** При продолжении процесса часто встречающиеся фрагменты и целые слова, такие как `'учи'`, `'учил'`, `'учили'`, `'училище'`, `'учитель'`, `'учит'`, `'ученикам'`, `'учебник'`, `'учебнику'`, `'по'`, также становятся отдельными токенами.

> **Важное примечание о префиксах:**  
> В реальных реализациях WordPiece (например, в BERT) для обозначения продолжения слова используется префикс `##`. Например, слово *"играющий"* может быть разбито на `["игра", "##ющий"]`. Это позволяет модели различать, где начинается новое слово, а где — продолжается предыдущее.  
> В нашем упрощённом примере мы опустили `##` для наглядности, но в реальных системах эта система критически важна для корректной обработки морфологии и редких слов.



**3. Токенизация Нового Текста с Использованием Обученного Словаря**

После завершения обучения и формирования финального словаря, токенизатор может обрабатывать любой новый текст. Процесс токенизации **одного слова** выглядит следующим образом:

1. Ищем **самый длинный подтокен** из словаря, который совпадает с началом (префиксом) слова.
2. Если такой подтокен найден, добавляем его в выходную последовательность и повторяем процесс для оставшейся части слова.
3. Если часть слова не может быть разбита на известные подтокены, она либо:
   - разбивается на отдельные символы (если они есть в словаре),
   - либо заменяется специальным токеном `[UNK]` (Unknown), если ни один фрагмент не найден.

Этот алгоритм называется **жадным разбиением по наибольшему префиксу** (greedy longest-match-first).


**Пример токенизации с гипотетическим Vfinal:**  
Текст: `"в училище учитель учит ученикам по новому учебнику"`

- **Слово "в":**  
  → "в" есть в словаре → `['в']`

- **Слово "училище":**  
  → "училище" есть в словаре → `['училище']`

- **Слово "учитель":**  
  → "учитель" есть в словаре → `['учитель']`

- **Слово "учит":**  
  → "учит" есть в словаре → `['учит']`

- **Слово "ученикам":**  
  → "ученикам" есть в словаре → `['ученикам']`

- **Слово "по":**  
  → "по" есть в словаре → `['по']`

- **Слово "новому":**  
  → "новому" есть в словаре → `['новому']`

- **Слово "учебнику":**  
  → "учебнику" есть в словаре → `['учебнику']`


**Финальная токенизация (с пробелами как отдельными токенами):**  
`['в', ' ', 'училище', ' ', 'учитель', ' ', 'учит', ' ', 'ученикам', ' ', 'по', ' ', 'новому', ' ', 'учебнику']`

> ⚠️ Примечание: В большинстве современных реализаций пробелы **не используются как отдельные токены**. Вместо этого они либо удаляются, либо заменяются на `##` при внутрисловном слиянии. Однако в нашем упрощённом примере пробел сохраняется для наглядности.



**Случай неизвестного слова:**  
Если бы в тексте появилось слово *"преподавательница"*, которого нет в словаре, но есть подтокены:
- `"препода"` → да
- `"##ватель"` → да
- `"##ница"` → да

То оно было бы токенизировано как:  
`['препода', '##ватель', '##ница']`

Если же ни один подтокен не найден (например, из-за опечатки), слово может быть заменено на `[UNK]`.


**4. Масштабирование WordPiece на Многопредметный Корпус**

При обучении на большом корпусе, состоящем из множества предложений или документов, алгоритм WordPiece адаптируется, чтобы сохранить лингвистическую корректность и избежать артефактов.

**Ключевые аспекты масштабирования:**

- **Обработка по границам предложений:**  
  Каждое предложение обрабатывается **независимо**. Подсчёт частот биграмм и слияние токенов **не пересекают границы предложений**. Это предотвращает бессмысленные слияния между последним словом одного предложения и первым — другого.

- **Специальные разделительные токены:**  
  Часто используются специальные маркеры:
  - `<s>` — начало предложения
  - `</s>` — конец предложения  
  Эти токены помогают модели понимать структуру текста и не допускают слияния через границы.

- **Запрет межпредложных слияний:**  
  Это принципиально важно. Например, если одно предложение заканчивается словом, оканчивающимся на `"ок"`, а следующее начинается с `"но"`, WordPiece **не будет** объединять `"ок"` и `"но"` в `"окно"`, потому что они находятся в разных предложениях.  
  Это гарантирует, что все формируемые токены имеют **лингвистический смысл в контексте одного предложения**.

- **Работа с большим корпусом:**  
  На практике WordPiece обучается на **гигантских корпусах** (например, весь Wikipedia). Процесс включает:
  - предварительную токенизацию на уровне слов,
  - построение начального символьного словаря,
  - десятки тысяч итераций слияния,
  - остановку по достижению целевого размера словаря (например, 30 000 токенов).



### **Пример: Обучение WordPiece на корпусе предложений**

Рассмотрим следующий корпус из трёх предложений:

1. `"Ученик учится в школе, а учитель учит ученика."`  
2. `"Новый студент изучает грамматику."`  
3. `"Учитель и студент читают книги."`



#### **Начальная настройка**

Перед обучением добавим специальные токены для обозначения границ предложений:  
- `<s>` — начало предложения  
- `</s>` — конец предложения

Также проведём предварительную обработку:
- Приведение к нижнему регистру
- Замена пробелов на символ `_` (для наглядности — в реальных реализациях используется пробел или `##`)
- Разделение на символы

**Токенизированный корпус (на начальном символьном уровне):**

```
<s> у _ ч _ е _ н _ и _ к _ _ у _ ч _ и _ т _ с _ я _ _ в _ _ ш _ к _ о _ л _ е _ , _ _ а _ _ у _ ч _ и _ т _ е _ л _ ь _ _ у _ ч _ и _ т _ _ у _ ч _ е _ н _ и _ к _ а _ . </s>
<s> _ н _ о _ в _ ы _ й _ _ с _ т _ у _ д _ е _ н _ т _ _ и _ з _ у _ ч _ а _ е _ т _ _ г _ р _ а _ м _ м _ а _ т _ и _ к _ у _ . </s>
<s> _ у _ ч _ и _ т _ е _ л _ ь _ _ и _ _ с _ т _ у _ д _ е _ н _ т _ _ ч _ и _ т _ а _ ю _ т _ _ к _ н _ и _ г _ и _ . </s>
```

> ⚠️ Примечание: В реальных реализациях (например, BERT) пробелы не заменяются на `_`, а обрабатываются как отдельные символы. Здесь `_` используется для наглядности.



#### **Итерации WordPiece на корпусе**

При обучении частоты биграмм считаются **внутри каждого предложения отдельно**, чтобы избежать слияний через границы. Затем частоты суммируются по всему корпусу.

**Пример подсчёта биграмм (гипотетический):**

- **Из предложения 1:**  
  - `('у', 'ч')`: 5  
  - `('и', 'к')`: 2  
  - `('_', 'у')`: 3  
  - `('а', '_')`: 2  
  - `('т', 'е')`: 1  

- **Из предложения 2:**  
  - `('н', 'о')`: 1  
  - `('с', 'т')`: 1  
  - `('т', 'у')`: 1  
  - `('г', 'р')`: 1  

- **Из предложения 3:**  
  - `('у', 'ч')`: 1  
  - `('т', 'е')`: 2  
  - `('ч', 'и')`: 1  
  - `('и', '_')`: 1  
  - `('к', 'н')`: 1  

**Суммарные частоты по корпусу:**
- `('у', 'ч')`: 5 + 1 = **6**
- `('_', 'у')`: 3 + 1 = **4**
- `('т', 'е')`: 1 + 2 = **3**
- `('и', 'к')`: 2
- `('с', 'т')`: 1 + 1 = **2**


#### **Решение об объединении**

На первой итерации пара `('у', 'ч')` имеет наивысшую частоту (6). Согласно формуле WordPiece:
$$
\text{Score}(A, B) = \frac{f(A, B)}{f(A) \times f(B)}
$$
она также, скорее всего, будет иметь высокую оценку, особенно если `у` и `ч` встречаются вместе значительно чаще, чем можно было бы ожидать случайно.

Мы объединяем `у` и `ч` в новый токен **`уч`**.

**Обновление корпуса:**  
Во всех предложениях заменяем последовательности `['у', 'ч']` на `['уч']`. Например:
- Было: `... _ у _ ч _ и _ т ...`  
- Стало: `... _ уч _ и _ т ...`

Процесс повторяется: пересчитываются частоты, выбирается следующая лучшая биграмма, выполняется слияние — и так до достижения целевого размера словаря.


#### **Почему этот подход важен**

- **Осмысленные токены:**  
  Поскольку слияния не пересекают границы предложений, исключаются артефакты вроде объединения конца одного предложения с началом другого (например, `"а_н"` из `"а."</s><s>н"`). Это гарантирует, что формируемые токены лингвистически осмысленны.

- **Целостность предложений:**  
  Сохранение границ помогает модели учитывать синтаксическую и семантическую структуру предложений — критично для задач, таких как машинный перевод, классификация или генерация.

- **Масштабируемость:**  
  Подход позволяет обрабатывать большие или потоковые корпусы, где не все данные доступны одновременно. Это делает WordPiece пригодным для промышленного использования.

- **Последовательная токенизация:**  
  Одинаковые слова в одинаковом контексте будут токенизированы одинаково, что обеспечивает стабильность и воспроизводимость.


### **5. Преимущества и недостатки WordPiece**

Хотя WordPiece широко используется (в BERT, DistilBERT и других моделях), у него есть как сильные стороны, так и ограничения.



#### **5.1. Преимущества WordPiece**

- **Эффективная обработка OOV-слов (Out-of-Vocabulary):**  
  WordPiece способен разбивать неизвестные слова на знакомые подслова. Например, *"программистом"* → `["программ", "##ист", "##ом"]`, даже если целое слово отсутствует в словаре. Это обеспечивает частичное понимание и устойчивость к лексической вариативности.

- **Контроль размера словаря:**  
  Размер словаря задаётся заранее (например, 30 000). Алгоритм итеративно выбирает наиболее "полезные" слияния, максимизируя правдоподобие текста. Это позволяет балансировать между компактностью и покрытием.

- **Гибридный подход:**  
  WordPiece сочетает преимущества символьной и словесной токенизации:
  - Частые слова (например, `"учитель"`) становятся одним токеном.
  - Редкие или сложные слова разбиваются на подслова.
  - Все входные данные могут быть представлены — нет "неизвестных" символов.

- **Универсальность (в варианте Byte-level WordPiece):**  
  Байтовый WordPiece (используется в моделях вроде RoBERTa) работает на уровне байтов, а не символов Unicode. Это делает его **инвариантным к кодировке**, способным обрабатывать любой текст (включая эмодзи, редкие символы), и **гарантирует отсутствие [UNK]**.



#### **5.2. Недостатки WordPiece**

- **Жадный и детерминированный алгоритм:**  
  На каждом шаге выбирается пара с максимальной оценкой. Такой подход не учитывает долгосрочных последствий и может привести к субоптимальной сегментации с точки зрения морфологии или семантики.

- **Отсутствие лингвистических знаний:**  
  WordPiece — статистический алгоритм. Он не "понимает" морфемы. Например, может разбить `"бежать"` на `["беж", "##ать"]`, хотя лингвистически более корректно — `["бег", "##ать"]`. Это снижает интерпретируемость и иногда — эффективность.

- **Вычислительная сложность:**  
  На ранних этапах обучения (при большом количестве символьных токенов) подсчёт частот всех биграмм требует значительных ресурсов. Для больших корпусов это может быть медленным.

- **Зависимость от начальной токенизации:**  
  Если перед WordPiece используется разбиение по словам, то ошибки на этом этапе (например, неудачная обработка знаков препинания) могут повлиять на итоговый словарь и сегментацию.

- **Неоднозначность сегментации:**  
  Одно и то же слово может быть разбито по-разному в зависимости от истории слияний. Например, `"ученик"` может стать `["уч", "##еник"]` или `["учени", "##к"]` — в зависимости от того, какие пары были выбраны ранее. Хотя на практике это редко вызывает проблемы, теоретически снижает согласованность.


### **Заключение**

WordPiece стал **краеугольным камнем** современных NLP-моделей. Его способность эффективно справляться с неизвестными словами, контролировать размер словаря и сохранять баланс между символами и словами сделала его выбором по умолчанию для таких моделей, как **BERT** и **DistilBERT**.

Однако его ограничения — жадность, отсутствие морфологического понимания — стимулировали разработку альтернатив:
- **SentencePiece** — обучается без предварительной токенизации по словам.
- **Unigram Language Model** — использует вероятностную модель для сегментации.
- **BoundlessBPE / BBPE** — улучшает морфологическую целостность.

Эта эволюция показывает: **нет универсального идеального токенизатора**. Выбор зависит от задачи, языка, требований к размеру модели и компромисса между эффективностью, точностью и универсальностью.


In [ ]:
import collections

class WordPieceTokenizer:
    def __init__(self):
        self.vocab = set()
        self.merges = []

    def preprocess_text(self, text):
        """
        Предварительная обработка текста: приведение к нижнему регистру и разбиение на слова.
        """
        # Удаляем знаки препинания и приводим к нижнему регистру
        processed_text = text.lower().replace('...', '').replace('.', '')
        # Разбиваем на слова по пробелам
        words = processed_text.split(' ')
        # Добавляем пробелы как отдельные токены между словами
        initial_tokens = []
        for i, word in enumerate(words):
            if word: # Убедимся, что слово не пустое
                initial_tokens.extend(list(word))
            if i < len(words) - 1:
                initial_tokens.append(' ') # Добавляем пробел между словами
        return initial_tokens, words # Возвращаем начальные токены и список слов для удобства

    def calculate_frequencies(self, tokens):
        """
        Подсчет частот отдельных токенов (униграмм) и биграмм в текущем корпусе.
        """
        unigram_freq = collections.defaultdict(int)
        bigram_freq = collections.defaultdict(int)

        for i in range(len(tokens)):
            unigram_freq[tokens[i]] += 1
            if i < len(tokens) - 1:
                bigram_freq[(tokens[i], tokens[i+1])] += 1
        return unigram_freq, bigram_freq

    def calculate_score(self, unigram_freq, bigram_freq):
        """
        Вычисление оценки слияния для всех возможных биграмм.
        Score(A, B) = frequency(AB) / (frequency(A) * frequency(B))
        """
        scores = {}
        for (token_a, token_b), freq_ab in bigram_freq.items():
            freq_a = unigram_freq[token_a]
            freq_b = unigram_freq[token_b]
            if freq_a > 0 and freq_b > 0: # Избегаем деления на ноль
                score = freq_ab / (freq_a * freq_b)
                scores[(token_a, token_b)] = score
        return scores

    def merge_tokens(self, tokens, best_bigram):
        """
        Слияние лучшей биграммы в корпусе.
        """
        merged_token = "".join(best_bigram)
        new_tokens = []
        i = 0
        while i < len(tokens):
            if i + 1 < len(tokens) and (tokens[i], tokens[i+1]) == best_bigram:
                new_tokens.append(merged_token)
                i += 2 # Пропускаем оба слитых токена
            else:
                new_tokens.append(tokens[i])
                i += 1
        return new_tokens

    def train(self, text, target_vocab_size=None, num_merges=None):
        """
        Обучение WordPiece токенизатора.
        Итеративно сливает токены до достижения целевого размера словаря
        или заданного количества слияний.
        """
        print(f"Исходный текст: '{text}'")
        current_tokens, _ = self.preprocess_text(text)

        # Начальный словарь состоит из всех уникальных символов
        self.vocab = set(current_tokens)
        print(f"\nШаг 1: Начальный словарь (V0) содержит {len(self.vocab)} токенов.")
        print(f"Начальный корпус: {current_tokens}")

        merges_count = 0
        while True:
            unigram_freq, bigram_freq = self.calculate_frequencies(current_tokens)
            scores = self.calculate_score(unigram_freq, bigram_freq)

            if not scores:
                print("\nНет больше биграмм для слияния. Остановка.")
                break

            # Находим биграмму с наивысшей оценкой
            best_bigram = max(scores, key=scores.get)
            best_score = scores[best_bigram]

            merged_token = "".join(best_bigram)

            # Критерий остановки: если новый токен уже в словаре
            if merged_token in self.vocab:
                # Если лучший токен уже существует, удалим его из scores, чтобы найти следующий лучший
                del scores[best_bigram]
                continue # Продолжаем поиск

            # Выводим информацию о текущем слиянии
            print(f"\nИтерация {merges_count + 1}:")
            print(f"  Лучшая биграмма для слияния: '{best_bigram[0]}' + '{best_bigram[1]}' -> '{merged_token}' (Score: {best_score:.4f})")

            # Выполняем слияние в корпусе
            current_tokens = self.merge_tokens(current_tokens, best_bigram)

            # Добавляем новый токен в словарь и сохраняем слияние
            self.vocab.add(merged_token)
            self.merges.append(best_bigram)
            merges_count += 1

            print(f"  Корпус после слияния: {current_tokens}")
            print(f"  Текущий размер словаря: {len(self.vocab)}")

            # Критерии остановки
            if target_vocab_size is not None and len(self.vocab) >= target_vocab_size:
                print(f"\nДостигнут целевой размер словаря ({target_vocab_size}). Остановка.")
                break
            if num_merges is not None and merges_count >= num_merges:
                print(f"\nДостигнуто заданное количество слияний ({num_merges}). Остановка.")
                break

        print(f"\nОбучение завершено. Итоговый словарь содержит {len(self.vocab)} токенов.")
        return sorted(list(self.vocab))

    def tokenize(self, text):
        """
        Токенизация нового текста с использованием обученного словаря WordPiece.
        """
        if not hasattr(self, 'vocab') or len(self.vocab) == 0:
            raise ValueError("Токенизатор не обучен. Сначала вызовите метод train().")

        print(f"\n--- Токенизация нового текста ---")
        print(f"Текст для токенизации: '{text}'")

        # Предварительная обработка текста для токенизации
        processed_text = text.lower().replace('...', '').replace('.', '')
        words = processed_text.split(' ')

        final_tokens = []
        for word in words:
            if not word: # Пропускаем пустые строки, если они возникли из-за множественных пробелов
                continue

            word_tokens = []
            remaining_word = word

            while remaining_word:
                found_match = False
                # Ищем самый длинный подтокен из словаря, который является префиксом оставшейся части слова
                for i in range(len(remaining_word), 0, -1):
                    subword = remaining_word[:i]

                    if subword in self.vocab:
                        word_tokens.append(subword)
                        remaining_word = remaining_word[i:]
                        found_match = True
                        break

                if not found_match:
                    # Если не удалось найти соответствующий токен, разбиваем на символы
                    # или используем токен [UNK]
                    if remaining_word[0] in self.vocab: # Если символ есть в словаре
                        word_tokens.append(remaining_word[0])
                    else: # Если символ даже не в начальном словаре
                        word_tokens.append('[UNK]')
                    remaining_word = remaining_word[1:]

            final_tokens.extend(word_tokens)
            final_tokens.append(' ') # Добавляем пробел между токенизированными словами

        # Удаляем последний пробел, если он есть
        if final_tokens and final_tokens[-1] == ' ':
            final_tokens.pop()

        print(f"Токенизированный текст: {final_tokens}")
        return final_tokens

    def get_vocab(self):
        """Возвращает текущий словарь токенов."""
        return sorted(list(self.vocab))

    def get_merges(self):
        """Возвращает список выполненных слияний."""
        return self.merges

# --- Пример использования ---
if __name__ == "__main__":
    text_to_train = "В училище учитель учит ученикам по новому учебнику"

    # Создаем экземпляр токенизатора
    tokenizer = WordPieceTokenizer()

    # Обучаем токенизатор
    final_vocab = tokenizer.train(text_to_train, num_merges=50)

    print(f"\nИтоговый словарь WordPiece: {final_vocab}")

    # Токенизируем тот же текст
    tokenizer.tokenize(text_to_train)

    # Пример токенизации нового слова
    new_text = "учительница"
    # Добавим '##ница' в словарь для демонстрации
    if 'ница' not in final_vocab:
        tokenizer.vocab.add('ница')
        tokenizer.vocab.add('##ница')
        final_vocab = tokenizer.get_vocab()

    print(f"\nИтоговый словарь WordPiece (с 'ница' для демонстрации): {final_vocab}")
    tokenizer.tokenize(new_text)

    # Демонстрация с упрощенным словарем
    simplified_tokenizer = WordPieceTokenizer()
    simplified_tokenizer.vocab = set(['у', 'ч', 'и', 'т', 'е', 'л', 'ь', 'н', 'ц', 'а', 'учитель', '##ница', '##тель', '##а'])
    print(f"\n--- Демонстрация токенизации подслов с упрощенным словарем ---")
    simplified_tokenizer.tokenize("учительница")



# **Униграмная Языковая Модель: Фундаментальные Принципы и Применение в НЛП**



### **1. Введение в Языковое Моделирование**

Языковое моделирование — одна из ключевых задач в обработке естественного языка (Natural Language Processing, NLP). Его основная цель — оценить вероятность появления последовательности слов, предсказать следующее слово в контексте или определить, насколько "естественно" звучит заданный фрагмент текста.

Языковые модели лежат в основе множества NLP-приложений:
- машинный перевод,
- распознавание речи,
- проверка орфографии и грамматики,
- генерация текста,
- информационный поиск,
- классификация и кластеризация текстов.

**Униграмная языковая модель (Unigram Language Model)** — это самая простая и базовая вероятностная модель языка. Она не учитывает порядок слов и контекст, но служит важным теоретическим и практическим фундаментом для понимания более сложных моделей, таких как биграммы, триграммы и нейросетевые языковые модели.

> ⚠️ **Важно:** Униграмная модель **не выполняет токенизацию**. Токенизация — это предварительный этап, в ходе которого текст разбивается на токены (слова, символы и т.д.). Униграмная модель работает **с уже токенизированными данными**, оценивая вероятности отдельных токенов.



### **2. Основные Концепции Униграмной Модели**

Центральное предположение униграмной модели — **независимость слов**. Модель считает, что каждое слово в тексте появляется независимо от других слов. То есть:
- Вероятность слова не зависит от предыдущих или последующих слов.
- Порядок слов в последовательности не имеет значения.

Формально, вероятность последовательности слов $w_1, w_2, \dots, w_n$ в униграмной модели вычисляется как:
$$
P(w_1, w_2, \dots, w_n) = \prod_{i=1}^{n} P(w_i)
$$
где $P(w_i)$ — вероятность отдельного слова $w_i$, оцениваемая по частоте его встречаемости в обучающем корпусе:
$$
P(w_i) = \frac{\text{count}(w_i)}{\text{total\_words}}
$$



#### **Пример:**

Рассмотрим предложение: *"кот сидит на коврике"*.  
Униграмная модель оценит его вероятность как:
$$
P(\text{кот}) \times P(\text{сидит}) \times P(\text{на}) \times P(\text{коврике})
$$
При этом модель не различает, является ли это осмысленным предложением или просто случайной перестановкой слов — например, *"на сидит коврике кот"* будет иметь ту же вероятность.


### **2.1. Области Применения Униграмных Моделей**

Несмотря на упрощённое предположение о независимости слов, униграмные модели находят практическое применение в задачах, где **частота слов** является ключевым фактором, а контекст — вторичным.

#### **• Базовая оценка вероятности текста**  
Униграмная модель может дать грубую оценку "естественности" текста. Например, в задачах определения авторства или жанра можно сравнивать распределения слов в разных текстах. Хотя это не учитывает синтаксис, оно может выявить стилистические особенности (например, частота местоимений или глаголов).

#### **• Тематическое моделирование (Topic Modeling)**  
Униграмные модели позволяют выявить наиболее частотные слова в документе. Эти слова часто коррелируют с основной темой. Например, высокая частота слов *"футбол"*, *"гол"*, *"матч"* указывает на спортивную тематику. Этот подход лежит в основе простых методов, таких как **TF-IDF** и **мешок слов (Bag-of-Words)**.

#### **• Спам-фильтрация**  
В классических фильтрах (например, на основе наивного байесовского классификатора) униграмные вероятности используются для оценки того, насколько "спамовым" является письмо. Если вероятность слов *"выигрыш"*, *"бесплатно"*, *"акция"* в спам-корпусе высока, то письмо с их высокой частотой будет классифицировано как спам.

#### **• Поиск информации (Information Retrieval)**  
В простых системах поиска (например, в ранних версиях TF-IDF) униграмные частоты слов используются для оценки релевантности документа запросу. Документ, содержащий больше слов из запроса, считается более релевантным — независимо от порядка слов.

#### **• Оценка сложности текста**  
Униграмные модели могут применяться для приблизительной оценки читабельности или сложности текста. Например:
- Высокая частота редких слов → более сложный текст.
- Большое количество уникальных слов (высокое лексическое разнообразие) → выше сложность.



### **3. Математические Формулы Униграмной Модели**



#### **3.1. Вероятность последовательности слов**

Пусть дана последовательность слов $W = (w_1, w_2, \dots, w_n)$.  
В униграмной языковой модели предполагается, что слова появляются **независимо** друг от друга. Это позволяет выразить вероятность всей последовательности как **произведение вероятностей отдельных слов**:

$$
P(W) = P(w_1, w_2, \dots, w_n) = \prod_{i=1}^{n} P(w_i)
$$

Где:
- $P(w_i)$ — вероятность слова $w_i$ в языковой модели,
- $\prod$ — знак произведения по всем словам в последовательности.

> ⚠️ **Важно:** Это предположение игнорирует как синтаксис, так и семантику. Например, последовательности *"кошка спит на диване"* и *"на спит диване кошка"* будут иметь одинаковую вероятность, если все слова одинаковы.



#### **3.2. Оценка вероятности слова: Максимальное правдоподобие (MLE)**

На практике вероятности слов $P(w)$ неизвестны и должны быть оценены на основе обучающего корпуса. Наиболее распространённый способ — **метод максимального правдоподобия (Maximum Likelihood Estimation, MLE)**.

Согласно MLE, вероятность слова $w$ оценивается как его **относительная частота** в обучающем корпусе:

$$
P(w) = \frac{\text{Count}(w)}{N}
$$

Где:
- $\text{Count}(w)$ — количество вхождений слова $w$ в обучающий корпус,
- $N = \sum_{w' \in V} \text{Count}(w')$ — общее количество слов в корпусе,
- $V$ — словарь корпуса (множество всех уникальных слов).

> **Интуиция:** Если слово *"кот"* встречается 10 раз в корпусе из 1000 слов, то его оценённая вероятность будет:
>$$
> P(\text{кот}) = \frac{10}{1000} = 0.01
>$$
> Это значение "максимизирует правдоподобие" обучающих данных — то есть делает наблюдаемый корпус наиболее вероятным при заданных параметрах модели.



#### **3.3. Проблема нулевых частот и сглаживание (Smoothing)**

**Критическая проблема MLE:**  
Если слово $w$ **не встречалось** в обучающем корпусе, то $\text{Count}(w) = 0$, и, следовательно, $P(w) = 0$.

Это приводит к **катастрофической ошибке**: любая последовательность, содержащая это слово, будет иметь нулевую вероятность:
$$
P(\dots, w, \dots) = \dots \times P(w) \times \dots = \dots \times 0 \times \dots = 0
$$
Модель становится **необобщаемой** — она не может обрабатывать ни одно новое слово, что делает её бесполезной в реальных условиях.



##### **Решение: Сглаживание (Smoothing)**

Чтобы решить эту проблему, применяются методы **сглаживания** — они перераспределяют часть вероятностной массы от наблюдаемых слов к ненаблюдаемым, гарантируя, что **все возможные слова имеют ненулевую вероятность**.



##### **Сглаживание по Лапласу (Laplace Smoothing / Add-One Smoothing)**

Один из самых простых методов — **сглаживание по Лапласу**, при котором к счётчику каждого слова (включая не встретившиеся) добавляется 1:

$$
P_{\text{Laplace}}(w) = \frac{\text{Count}(w) + 1}{N + |V|}
$$

Где:
- $\text{Count}(w)$ — частота слова $w$ в корпусе,
- $N$ — общее количество слов в корпусе,
- $|V|$ — размер словаря (число уникальных слов в корпусе).

> ⚠️ **Примечание:** В знаменателе добавляется $|V|$, потому что мы "добавляем по единице" для каждого из $|V|$ возможных слов.



##### **Интуиция сглаживания по Лапласу:**

Представим, что **каждое возможное слово из словаря** встретилось в корпусе **хотя бы один раз**. Это искусственно "расширяет" корпус и предотвращает нулевые вероятности.

> **Пример:**  
> Корпус: `["кот", "спит", "кот", "еда"]`  
> - $N = 4$, $|V| = 4$ (слова: "кот", "спит", "еда", и предполагаемое "новое")  
> - $P_{\text{MLE}}(\text{собака}) = 0$  
> - $P_{\text{Laplace}}(\text{собака}) = \frac{0 + 1}{4 + 4} = \frac{1}{8} = 0.125$

Теперь даже незнакомые слова имеют шанс.



##### **Недостатки сглаживания по Лапласу:**
- Сильно **искажает вероятности** частотных слов, особенно при большом $|V|$.
- Не подходит для больших словарей (например, в языковых моделях с десятками тысяч слов), так как "перераспределяет слишком много" вероятности.
- Поэтому на практике чаще используются более продвинутые методы: **Good-Turing**, **Witten-Bell**, **Kneser-Ney smoothing**.





### **4. Конкретный Пример Применения Униграмной Модели**

Рассмотрим применение униграмной языковой модели к следующему тексту:  
**"Ученики пишут диктант. Учитель диктует. Мы пишем, он пишет, они пишут. Пишите внимательно."**



#### **4.1. Шаг 1: Токенизация и подготовка корпуса**

Перед построением модели необходимо предварительно обработать текст:
- Привести к нижнему регистру,
- Удалить знаки препинания,
- Разделить на слова (токены).

**Исходный текст:**  
"Ученики пишут диктант. Учитель диктует. Мы пишем, он пишет, они пишут. Пишите внимательно."

**Токенизированный корпус (после обработки):**  
`['ученики', 'пишут', 'диктант', 'учитель', 'диктует', 'мы', 'пишем', 'он', 'пишет', 'они', 'пишут', 'пишите', 'внимательно']`

**Общее количество слов в корпусе (N):**  
$$
N = 13
$$



#### **4.2. Шаг 2: Подсчёт частот слов**

Составим таблицу частот для каждого уникального слова:

| Слово         | Частота $\text{Count}(w)$ |
|---------------|-------------------------------|
| ученики       | 1                             |
| пишут         | 2                             |
| диктант       | 1                             |
| учитель       | 1                             |
| диктует       | 1                             |
| мы            | 1                             |
| пишем         | 1                             |
| он            | 1                             |
| пишет         | 1                             |
| они           | 1                             |
| пишите        | 1                             |
| внимательно   | 1                             |

> ⚠️ Обратите внимание: слово **"пишут"** встречается дважды — в начале и в середине текста.



#### **4.3. Шаг 3: Определение словаря и его размера**

**Словарь $V$** — множество всех уникальных слов в корпусе:  
$$
V = \{ \text{'ученики'}, \text{'пишут'}, \text{'диктант'}, \text{'учитель'}, \text{'диктует'}, \text{'мы'}, \text{'пишем'}, \text{'он'}, \text{'пишет'}, \text{'они'}, \text{'пишите'}, \text{'внимательно'} \}
$$

**Размер словаря $|V|$:**  
$$
|V| = 12
$$


#### **4.4. Шаг 4: Расчёт вероятностей слов (MLE)**

Используем метод максимального правдоподобия (MLE):  
$$
P(w) = \frac{\text{Count}(w)}{N} = \frac{\text{Count}(w)}{13}
$$

Вычислим вероятности:

| Слово         | $\text{Count}(w)$ | $P(w)$             |
|---------------|------------------------|-------------------------|
| ученики       | 1                      | $\frac{1}{13} \approx 0.0769$ |
| пишут         | 2                      | $\frac{2}{13} \approx 0.1538$ |
| диктант       | 1                      | $\frac{1}{13} \approx 0.0769$ |
| учитель       | 1                      | $\frac{1}{13} \approx 0.0769$ |
| диктует       | 1                      | $\frac{1}{13} \approx 0.0769$ |
| мы            | 1                      | $\frac{1}{13} \approx 0.0769$ |
| пишем         | 1                      | $\frac{1}{13} \approx 0.0769$ |
| он            | 1                      | $\frac{1}{13} \approx 0.0769$ |
| пишет         | 1                      | $\frac{1}{13} \approx 0.0769$ |
| они           | 1                      | $\frac{1}{13} \approx 0.0769$ |
| пишите        | 1                      | $\frac{1}{13} \approx 0.0769$ |
| внимательно   | 1                      | $\frac{1}{13} \approx 0.0769$ |

> Слово **"пишут"** имеет наибольшую вероятность ($\approx 0.1538$) из-за своей высокой частоты.



#### **4.5. Шаг 5: Расчёт вероятности предложения**

Оценим вероятность всей последовательности (все слова из корпуса в порядке их появления):

$$
P(\text{предложение}) = \prod_{i=1}^{13} P(w_i)
$$

Распишем произведение:
$$
P = P(\text{ученики}) \times P(\text{пишут}) \times P(\text{диктант}) \times P(\text{учитель}) \times P(\text{диктует}) \times P(\text{мы}) \times P(\text{пишем}) \times P(\text{он}) \times P(\text{пишет}) \times P(\text{они}) \times P(\text{пишут}) \times P(\text{пишите}) \times P(\text{внимательно})
$$

Подставим значения:
$$
P = \left( \frac{1}{13} \right) \times \left( \frac{2}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{2}{13} \right) \times \left( \frac{1}{13} \right) \times \left( \frac{1}{13} \right)
$$

Упростим:
$$
P = \frac{1 \times 2 \times 1 \times 1 \times 1 \times 1 \times 1 \times 1 \times 1 \times 1 \times 2 \times 1 \times 1}{13^{13}} = \frac{4}{13^{13}}
$$

Вычислим:
$$
13^{13} \approx 3.028 \times 10^{14}, \quad \Rightarrow P \approx \frac{4}{3.028 \times 10^{14}} \approx 1.32 \times 10^{-14}
$$

$$
\boxed{P(\text{предложение}) \approx 1.32 \times 10^{-14}}
$$

> ⚠️ Несмотря на то что текст осмысленный, его вероятность крайне мала — это типично для униграмных моделей из-за умножения множества малых вероятностей.



#### **4.6. Пример со сглаживанием по Лапласу**

Предположим, мы хотим оценить вероятность слова **"школа"**, которого **нет в обучающем корпусе**.

- **Без сглаживания (MLE):**  
$$
  P(\text{школа}) = \frac{\text{Count}(\text{школа})}{N} = \frac{0}{13} = 0
$$
  Это означает, что любое предложение, содержащее слово "школа", будет иметь нулевую вероятность.

- **Со сглаживанием по Лапласу:**  
  Используем формулу:
$$
  P_{\text{Laplace}}(w) = \frac{\text{Count}(w) + 1}{N + |V|}
$$
  Подставим значения:
  - $\text{Count}(\text{школа}) = 0$
  - $N = 13$
  - $|V| = 12$

$$
  P_{\text{Laplace}}(\text{школа}) = \frac{0 + 1}{13 + 12} = \frac{1}{25} = 0.04
$$

$$
\boxed{P_{\text{Laplace}}(\text{школа}) = 0.04}
$$

Теперь слово "школа" имеет **ненулевую вероятность**, и модель может обрабатывать новые, ранее не встречавшиеся слова. Это делает её **более робастной** в реальных условиях.



### **5. Преимущества и Недостатки Униграмной Модели**

Несмотря на свою простоту, униграмная языковая модель обладает как сильными сторонами, так и серьёзными ограничениями. Понимание этих аспектов помогает оценить её место в иерархии языковых моделей и определить области, где она остаётся полезной.


#### **5.1. Преимущества**

- **Простота и интуитивность**  
  Униграмная модель основана на простой идее: частота слова напрямую отражает его вероятность. Это делает модель крайне простой для понимания, объяснения и реализации. Она служит отличной **обучающей платформой** для изучения основ языкового моделирования.

- **Вычислительная эффективность**  
  Обучение модели сводится к подсчёту частот слов — операции со сложностью $O(N)$, где $N$ — размер корпуса. Использование модели (оценка вероятности последовательности) также требует лишь умножения вероятностей. Это делает униграмные модели **очень быстрыми** и пригодными для обработки **огромных объёмов данных** или встраивания в системы с жёсткими требованиями к производительности.

- **Эффективная оценка частот слов**  
  Модель идеально подходит для задач, где **ключевую роль играют частоты отдельных слов**, а порядок не важен. Это включает:
  - тематическое моделирование,
  - фильтрацию спама,
  - предварительный анализ текста,
  - построение словарей и оценку лексического разнообразия.

- **Отсутствие проблемы разреженности (в сравнении с высокими N-граммами)**  
  В отличие от биграмм или триграмм, где многие возможные комбинации слов отсутствуют в корпусе (что приводит к разреженности данных), униграмная модель работает с отдельными словами. Для **частотных слов** проблема разреженности минимальна, так как они, как правило, наблюдаются в корпусе.



#### **5.2. Недостатки**

- **Полное игнорирование контекста и порядка слов**  
  Это — **главный и наиболее критичный недостаток**. Униграмная модель не учитывает ни синтаксис, ни семантику. Она не различает осмысленные и бессмысленные последовательности, если они содержат одни и те же слова.  
  Например, предложения *"собака укусила человека"* и *"человек укусил собаку"* будут иметь **одинаковую вероятность**, хотя их смысл диаметрально противоположен.

- **Нереалистичное предположение о независимости слов**  
  Естественный язык — это последовательность взаимозависимых слов. Вероятность появления слова *"кофе"* сильно возрастает после *"я пью"*, но почти нулевая после *"я лаю"*. Униграмная модель **не может улавливать такие зависимости**, что делает её слишком грубой для задач, требующих понимания смысла.

- **Проблема нулевых вероятностей (OOV-слова)**  
  Без применения сглаживания любое слово, не встретившееся в обучающем корпусе, получает вероятность 0. Это означает, что **любое предложение с новым словом** будет иметь нулевую вероятность — модель необобщаема. Хотя сглаживание решает эту проблему частично, оно вносит искажения в распределение.

- **Ограниченная применимость в современном НЛП**  
  Из-за отсутствия контекстной чувствительности униграмные модели **не подходят** для большинства современных задач, таких как:
  - генерация текста,
  - машинный перевод,
  - вопросно-ответные системы,
  - распознавание речи,
  - суммаризация.  
  Эти задачи требуют понимания последовательности, зависимостей и смысла — чего униграмная модель дать не может.


### **Заключение**

Униграмная языковая модель — это **упрощённая, но важная абстракция**, которая демонстрирует базовые принципы вероятностного моделирования в НЛП. Её сила — в простоте, скорости и интерпретируемости. Её слабость — в полном игнорировании контекста.

Однако именно эти ограничения стали **двигателем прогресса** в области языкового моделирования:
- от униграммы → к биграммам и триграммам (с учётом контекста),
- от n-грамм → к нейросетевым моделям (RNN, LSTM, трансформерам),
- от частотных подходов → к распределённым представлениям слов (word embeddings, BERT и др.).

Таким образом, униграмная модель — не инструмент для современных сложных систем, а **необходимая ступень на пути к ним**. Она помогает понять, **почему контекст важен**, и почему более сложные модели стали неизбежными в эпоху глубокого обучения.


In [ ]:
import collections
import re
import math

class UnigramLanguageModel:
    """
    Реализация Униграмной Языковой Модели.
    Эта модель предполагает, что вероятность каждого слова в последовательности
    не зависит от других слов.
    """
    def __init__(self):
        self.word_counts = collections.defaultdict(int) # Счетчик вхождений каждого слова
        self.total_words = 0 # Общее количество слов в корпусе
        self.vocabulary = set() # Множество уникальных слов (словарь)

    def _tokenize(self, text):
        """
        Приватный метод для токенизации текста.
        Приводит текст к нижнему регистру и разбивает на слова, удаляя знаки препинания.
        :param text: Входная строка текста.
        :return: Список токенов (слов).
        """
        # Приводим к нижнему регистру
        text = text.lower()
        # Удаляем знаки препинания (кроме тех, что могут быть частью слова, но для простоты здесь удаляем все)
        # Использование регулярного выражения для извлечения только буквенных последовательностей
        tokens = re.findall(r'\b[а-яё]+\b', text)
        return tokens

    def train(self, corpus_text):
        """
        Обучает униграмную языковую модель на заданном текстовом корпусе.
        Подсчитывает частоты слов и формирует словарь.
        :param corpus_text: Строка, представляющая обучающий корпус.
        """
        print("--- Начало обучения Униграмной Модели ---")
        tokens = self._tokenize(corpus_text)

        if not tokens:
            print("Предупреждение: Корпус пуст или не содержит слов после токенизации.")
            return

        self.total_words = len(tokens)

        for word in tokens:
            self.word_counts[word] += 1
            self.vocabulary.add(word)

        print(f"Обучение завершено. Общее количество слов (N): {self.total_words}")
        print(f"Размер словаря (|V|): {len(self.vocabulary)}")
        print(f"Частоты слов: {dict(self.word_counts)}")
        print("--- Обучение завершено ---")

    def get_word_probability(self, word, smoothing=None):
        """
        Возвращает вероятность слова P(w).
        :param word: Слово, для которого нужно рассчитать вероятность.
        :param smoothing: Метод сглаживания ('laplace' для сглаживания по Лапласу, None для MLE).
        :return: Вероятность слова.
        """
        # Если модель не обучена или корпус пуст
        if self.total_words == 0:
            print("Ошибка: Модель не обучена или корпус пуст. Невозможно рассчитать вероятность.")
            return 0.0

        count_w = self.word_counts[word] # Count(w)

        if smoothing == 'laplace':
            # Сглаживание по Лапласу: (Count(w) + 1) / (N + |V|)
            probability = (count_w + 1) / (self.total_words + len(self.vocabulary))
            # print(f"P_Laplace('{word}') = ({count_w} + 1) / ({self.total_words} + {len(self.vocabulary)}) = {probability:.4f}")
        else:
            # Метод максимального правдоподобия (MLE): Count(w) / N
            if count_w == 0:
                # print(f"P_MLE('{word}') = 0 (слово не найдено в корпусе)")
                return 0.0 # Если слово не найдено, вероятность 0 без сглаживания
            probability = count_w / self.total_words
            # print(f"P_MLE('{word}') = {count_w} / {self.total_words} = {probability:.4f}")

        return probability

    def get_sequence_probability(self, sequence_text, smoothing=None):
        """
        Возвращает вероятность последовательности слов P(W).
        :param sequence_text: Строка, представляющая последовательность слов.
        :param smoothing: Метод сглаживания ('laplace' для сглаживания по Лапласу, None для MLE).
        :return: Вероятность последовательности.
        """
        print(f"\n--- Расчет вероятности последовательности: '{sequence_text}' ---")
        tokens = self._tokenize(sequence_text)

        if not tokens:
            print("Предупреждение: Последовательность пуста или не содержит слов после токенизации.")
            return 0.0

        sequence_probability = 1.0
        probabilities_list = []

        for word in tokens:
            p_word = self.get_word_probability(word, smoothing)
            probabilities_list.append(p_word)
            sequence_probability *= p_word

            # Если хоть одно слово имеет нулевую вероятность без сглаживания,
            # вся последовательность будет иметь нулевую вероятность.
            if smoothing != 'laplace' and p_word == 0:
                print(f"Слово '{word}' имеет нулевую вероятность. Вероятность всей последовательности = 0.")
                return 0.0 # Оптимизация: если P(w)=0, то произведение будет 0

        print(f"Токены последовательности: {tokens}")
        print(f"Вероятности отдельных слов: {probabilities_list}")
        print(f"Итоговая вероятность последовательности: {sequence_probability}")
        print("--- Расчет завершен ---")
        return sequence_probability

# --- Пример использования ---
if __name__ == "__main__":
    corpus_example = "Ученики пишут диктант. Учитель диктует. Мы пишем, он пишет, они пишут. Пишите внимательно."

    # Создаем экземпляр модели
    unigram_model = UnigramLanguageModel()

    # Обучаем модель на корпусе
    unigram_model.train(corpus_example)

    print("\n--- Расчет вероятностей отдельных слов (MLE) ---")
    words_to_check = ['пишут', 'учитель', 'школа', 'мы']
    for word in words_to_check:
        prob = unigram_model.get_word_probability(word, smoothing=None)
        print(f"P_MLE('{word}') = {prob:.4f}")

    print("\n--- Расчет вероятностей отдельных слов (Лаплас) ---")
    for word in words_to_check:
        prob = unigram_model.get_word_probability(word, smoothing='laplace')
        print(f"P_Laplace('{word}') = {prob:.4f}")

    # Расчет вероятности предложения (без сглаживания)
    sentence_mle = "Ученики пишут диктант"
    prob_mle = unigram_model.get_sequence_probability(sentence_mle, smoothing=None)
    print(f"Вероятность предложения (MLE): {prob_mle:.10f}")

    sentence_mle_oov = "Ученики пишут школа" # Содержит OOV слово 'школа'
    prob_mle_oov = unigram_model.get_sequence_probability(sentence_mle_oov, smoothing=None)
    print(f"Вероятность предложения с OOV (MLE): {prob_mle_oov:.10f}")

    # Расчет вероятности предложения (со сглаживанием по Лапласу)
    sentence_laplace = "Ученики пишут диктант"
    prob_laplace = unigram_model.get_sequence_probability(sentence_laplace, smoothing='laplace')
    print(f"Вероятность предложения (Лаплас): {prob_laplace:.10f}")

    sentence_laplace_oov = "Ученики пишут школа" # Содержит OOV слово 'школа'
    prob_laplace_oov = unigram_model.get_sequence_probability(sentence_laplace_oov, smoothing='laplace')
    print(f"Вероятность предложения с OOV (Лаплас): {prob_laplace_oov:.10f}")

    # Пример из документации: "Ученики пишут диктант Учитель диктует Мы пишем он пишет они пишут Пишите внимательно"
    full_sentence = "Ученики пишут диктант Учитель диктует Мы пишем он пишет они пишут Пишите внимательно"
    prob_full_mle = unigram_model.get_sequence_probability(full_sentence, smoothing=None)
    print(f"Вероятность полного предложения (MLE): {prob_full_mle:.15f}")

    prob_full_laplace = unigram_model.get_sequence_probability(full_sentence, smoothing='laplace')
    print(f"Вероятность полного предложения (Лаплас): {prob_full_laplace:.15f}")

In [ ]:
import sentencepiece as spm
import os

class SentencePieceTokenizer:
    """
    Класс для обучения, сохранения и использования модели токенизации SentencePiece.
    """

    def __init__(self, model_prefix="my_sentencepiece_model"):
        """
        Инициализирует токенизатор.

        Args:
            model_prefix (str): Префикс для файлов модели и словаря.
                                 Будут созданы файлы <model_prefix>.model и <model_prefix>.vocab.
        """
        self.model_prefix = model_prefix
        self.sp_processor = None
        self.model_path = f"{model_prefix}.model"
        self.vocab_path = f"{model_prefix}.vocab"

    def train(self, input_file, vocab_size=8000, model_type="unigram", character_coverage=1.0, num_threads=os.cpu_count()):
        """
        Обучает модель SentencePiece на заданном текстовом корпусе.

        Args:
            input_file (str): Путь к файлу с необработанным текстовым корпусом.
                              Каждый абзац или предложение должен быть на новой строке.
            vocab_size (int): Желаемый размер словаря подслов.
            model_type (str): Тип алгоритма токенизации ('bpe' или 'unigram').
            character_coverage (float): Доля символов, которые должны быть покрыты моделью (от 0.0 до 1.0).
            num_threads (int): Количество потоков для параллельного обучения.
        """
        print(f"Начинаем обучение модели SentencePiece с параметрами:")
        print(f"  Входной файл: {input_file}")
        print(f"  Префикс модели: {self.model_prefix}")
        print(f"  Размер словаря: {vocab_size}")
        print(f"  Тип модели: {model_type}")
        print(f"  Покрытие символов: {character_coverage}")
        print(f"  Количество потоков: {num_threads}")

        try:
            spm.SentencePieceTrainer.train(
                input=input_file,
                model_prefix=self.model_prefix,
                vocab_size=vocab_size,
                model_type=model_type,
                character_coverage=character_coverage,
                num_threads=num_threads
            )
            print(f"Обучение завершено. Модель сохранена как {self.model_path} и {self.vocab_path}")
            self.load_model() # Загружаем обученную модель сразу после обучения
        except Exception as e:
            print(f"Ошибка при обучении модели: {e}")
            raise

    def load_model(self):
        """
        Загружает обученную модель SentencePiece.
        """
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Файл модели не найден: {self.model_path}. Пожалуйста, сначала обучите модель.")
        try:
            self.sp_processor = spm.SentencePieceProcessor(model_file=self.model_path)
            print(f"Модель SentencePiece успешно загружена из {self.model_path}")
        except Exception as e:
            print(f"Ошибка при загрузке модели: {e}")
            raise

    def tokenize_as_pieces(self, text):
        """
        Токенизирует текст в подслова (pieces).

        Args:
            text (str): Входной текст для токенизации.

        Returns:
            list: Список подслов.
        """
        if self.sp_processor is None:
            raise RuntimeError("Модель не загружена. Пожалуйста, обучите или загрузите модель.")
        return self.sp_processor.encode_as_pieces(text)

    def tokenize_as_ids(self, text):
        """
        Токенизирует текст в ID подслов.

        Args:
            text (str): Входной текст для токенизации.

        Returns:
            list: Список ID подслов.
        """
        if self.sp_processor is None:
            raise RuntimeError("Модель не загружена. Пожалуйста, обучите или загрузите модель.")
        return self.sp_processor.encode_as_ids(text)

    def decode_pieces(self, pieces):
        """
        Декодирует список подслов обратно в текст.

        Args:
            pieces (list): Список подслов.

        Returns:
            str: Восстановленный текст.
        """
        if self.sp_processor is None:
            raise RuntimeError("Модель не загружена. Пожалуйста, обучите или загрузите модель.")
        return self.sp_processor.decode_pieces(pieces)

    def decode_ids(self, ids):
        """
        Декодирует список ID подслов обратно в текст.

        Args:
            ids (list): Список ID подслов.

        Returns:
            str: Восстановленный текст.
        """
        if self.sp_processor is None:
            raise RuntimeError("Модель не загружена. Пожалуйста, обучите или загрузите модель.")
        return self.sp_processor.decode_ids(ids)

    def get_vocab(self):
        """
        Возвращает словарь модели (подслова и их ID).

        Returns:
            dict: Словарь, где ключ - подслово, значение - его ID.
        """
        if self.sp_processor is None:
            raise RuntimeError("Модель не загружена. Пожалуйста, обучите или загрузите модель.")
        return {self.sp_processor.id_to_piece(i): i for i in range(self.sp_processor.get_piece_size())}

# --- Пример использования класса ---
if __name__ == "__main__":
    # 1. Создаем тестовый файл для обучения
    training_data = [
        "SentencePiece - это мощный инструмент для токенизации подслов, разработанный Google.",
        "Он позволяет работать с необработанным текстом и является языково-агностическим.",
        "Токенизация - важный шаг в обработке естественного языка.",
        "Примеры использования включают машинный перевод и большие языковые модели."
    ]
    input_file_name = "train_corpus.txt"
    with open(input_file_name, "w", encoding="utf-8") as f:
        for line in training_data:
            f.write(line + "\n")

    # 2. Инициализируем токенизатор
    tokenizer = SentencePieceTokenizer(model_prefix="my_nlp_tokenizer")

    # 3. Обучаем модель
    try:
        # Изменено: vocab_size уменьшен до 91, чтобы соответствовать размеру корпуса
        tokenizer.train(input_file=input_file_name, vocab_size=91, model_type="unigram")

        # 4. Используем обученную модель для токенизации
        text_to_tokenize = "SentencePiece упрощает NLP задачи в области машинного перевода."
        pieces = tokenizer.tokenize_as_pieces(text_to_tokenize)
        ids = tokenizer.tokenize_as_ids(text_to_tokenize)

        print(f"\nИсходный текст: {text_to_tokenize}")
        print(f"Токены (pieces): {pieces}")
        print(f"ID токенов (ids): {ids}")

        # 5. Декодируем токены обратно в текст
        decoded_text_from_pieces = tokenizer.decode_pieces(pieces)
        decoded_text_from_ids = tokenizer.decode_ids(ids)

        print(f"Декодированный текст из pieces: {decoded_text_from_pieces}")
        print(f"Декодированный текст из ids: {decoded_text_from_ids}")

        # 6. Получаем словарь
        model_vocab = tokenizer.get_vocab()
        print(f"\nРазмер словаря: {len(model_vocab)}")
        # print(f"Несколько примеров из словаря: {list(model_vocab.items())[:20]}")

    except Exception as e:
        print(f"Произошла ошибка в процессе: {e}")
    finally:
        # Очистка: удаляем созданные файлы модели и словаря
        if os.path.exists(input_file_name):
            os.remove(input_file_name)
        if os.path.exists(tokenizer.model_path):
            os.remove(tokenizer.model_path)
        if os.path.exists(tokenizer.vocab_path):
            os.remove(tokenizer.vocab_path)
        print("\nТестовые файлы удалены.")


import React, { useState, useEffect } from 'react';

// Основной компонент приложения
const App = () => {
  // Состояние для текста, используемого для "обучения" модели
  const [trainingText, setTrainingText] = useState(
    "SentencePiece - это мощный инструмент для токенизации подслов, разработанный Google. Он позволяет работать с необработанным текстом и является языково-агностическим."
  );
  // Состояние для текста, который будет токенизирован
  const [inputText, setInputText] = useState(
    "SentencePiece упрощает NLP задачи."
  );
  // Состояние для имитации словаря
  const [vocab, setVocab] = useState([]);
  // Состояние для токенизированных частей
  const [tokenizedPieces, setTokenizedPieces] = useState([]);
  // Состояние для ID токенов
  const [tokenizedIds, setTokenizedIds] = useState([]);
  // Состояние для индикатора загрузки обучения
  const [isTraining, setIsTraining] = useState(false);
  // Состояние для индикатора загрузки токенизации
  const [isTokenizing, setIsTokenizing] = useState(false);
  // Состояние для сообщения об ошибке
  const [error, setError] = useState('');

  // Функция для имитации обучения модели SentencePiece
  const handleTrainModel = () => {
    setError('');
    setIsTraining(true);
    // Имитация задержки обучения
    setTimeout(() => {
      try {
        // Очень упрощенная имитация обучения:
        // Разбиваем текст на слова, затем на более мелкие части,
        // чтобы имитировать подслова.
        // В реальной SentencePiece модели используется BPE или Unigram.
        const words = trainingText.toLowerCase().split(/\s+/).filter(Boolean);
        const uniqueSubwords = new Set();

        // Добавляем пробел как специальный токен
        uniqueSubwords.add(' ');

        words.forEach(word => {
          // Добавляем слово целиком
          uniqueSubwords.add(word);
          // Добавляем части слова для имитации подслов
          for (let i = 1; i <= word.length; i++) {
            uniqueSubwords.add(word.substring(0, i));
            uniqueSubwords.add(word.substring(word.length - i));
          }
          // Добавляем биграммы и триграммы
          for (let i = 0; i < word.length - 1; i++) {
            uniqueSubwords.add(word.substring(i, i + 2));
          }
          for (let i = 0; i < word.length - 2; i++) {
            uniqueSubwords.add(word.substring(i, i + 3));
          }
        });

        // Сортируем для детерминированного ID
        const sortedVocab = Array.from(uniqueSubwords).sort((a, b) => b.length - a.length || a.localeCompare(b));
        setVocab(sortedVocab);
        console.log("Обученный словарь (имитация):", sortedVocab);
      } catch (e) {
        setError("Ошибка при имитации обучения модели: " + e.message);
        console.error("Ошибка при имитации обучения модели:", e);
      } finally {
        setIsTraining(false);
      }
    }, 1500); // Имитация времени обучения
  };

  // Функция для имитации токенизации текста
  const handleTokenizeText = () => {
    setError('');
    if (vocab.length === 0) {
      setError("Пожалуйста, сначала обучите модель.");
      return;
    }
    setIsTokenizing(true);
    // Имитация задержки токенизации
    setTimeout(() => {
      try {
        const textToTokenize = inputText.toLowerCase();
        let currentText = textToTokenize;
        const pieces = [];
        const ids = [];
        let idCounter = 0; // Для присвоения уникальных ID в демонстрации

        // Добавляем пробел в начало, как это делает SentencePiece
        currentText = ' ' + currentText;

        while (currentText.length > 0) {
          let foundMatch = false;
          // Ищем самый длинный совпадающий токен из словаря
          for (const token of vocab) {
            if (currentText.startsWith(token)) {
              pieces.push(token);
              // Присваиваем простой ID для демонстрации
              ids.push(vocab.indexOf(token) !== -1 ? vocab.indexOf(token) : idCounter++);
              currentText = currentText.substring(token.length);
              foundMatch = true;
              break; // Нашли самый длинный, переходим к следующему участку текста
            }
          }
          if (!foundMatch && currentText.length > 0) {
            // Если не нашли совпадения, берем один символ (имитация UNK или символьного токена)
            const char = currentText[0];
            pieces.push(char);
            ids.push(vocab.indexOf(char) !== -1 ? vocab.indexOf(char) : idCounter++);
            currentText = currentText.substring(1);
          }
          // Если currentText стал пустым, выходим из цикла
          if (currentText.length === 0 && !foundMatch) {
            break;
          }
        }
        setTokenizedPieces(pieces);
        setTokenizedIds(ids);
        console.log("Токенизированные части (имитация):", pieces);
        console.log("ID токенов (имитация):", ids);
      } catch (e) {
        setError("Ошибка при имитации токенизации: " + e.message);
        console.error("Ошибка при имитации токенизации:", e);
      } finally {
        setIsTokenizing(false);
      }
    }, 1000); // Имитация времени токенизации
  };

  // Функция для очистки всех результатов
  const handleClear = () => {
    setVocab([]);
    setTokenizedPieces([]);
    setTokenizedIds([]);
    setError('');
  };

  return (
    <div className="min-h-screen bg-gray-100 p-4 font-sans flex flex-col items-center">
      <div className="w-full max-w-4xl bg-white rounded-lg shadow-xl p-6 space-y-6">
        <h1 className="text-3xl font-bold text-center text-gray-800 mb-6">
          Демонстрация токенизации с использованием SentencePiece (имитация)
        </h1>

        {error && (
          <div className="bg-red-100 border border-red-400 text-red-700 px-4 py-3 rounded relative" role="alert">
            <strong className="font-bold">Ошибка:</strong>
            <span className="block sm:inline"> {error}</span>
          </div>
        )}

        {/* Секция обучения модели */}
        <div className="bg-blue-50 p-4 rounded-lg border border-blue-200">
          <h2 className="text-xl font-semibold text-blue-800 mb-3">1. Обучение модели SentencePiece</h2>
          <p className="text-sm text-gray-600 mb-4">
            Введите текст для "обучения" модели токенизации. В реальной SentencePiece модели обучение происходит на большом корпусе данных.
            Эта демонстрация имитирует процесс, создавая упрощенный словарь подслов.
          </p>
          <textarea
            className="w-full p-3 border border-gray-300 rounded-md focus:ring-blue-500 focus:border-blue-500 text-gray-700"
            rows="6"
            placeholder="Введите текст для обучения модели..."
            value={trainingText}
            onChange={(e) => setTrainingText(e.target.value)}
            disabled={isTraining}
          ></textarea>
          <button
            onClick={handleTrainModel}
            className={`mt-4 w-full py-3 px-6 rounded-md text-white font-semibold transition-all duration-300 ease-in-out
              ${isTraining ? 'bg-blue-300 cursor-not-allowed' : 'bg-blue-600 hover:bg-blue-700 focus:outline-none focus:ring-2 focus:ring-blue-500 focus:ring-offset-2'}`}
            disabled={isTraining}
          >
            {isTraining ? 'Обучение...' : 'Обучить модель'}
          </button>
        </div>

        {/* Секция токенизации текста */}
        <div className="bg-green-50 p-4 rounded-lg border border-green-200">
          <h2 className="text-xl font-semibold text-green-800 mb-3">2. Токенизация текста</h2>
          <p className="text-sm text-gray-600 mb-4">
            Введите текст для токенизации с использованием "обученной" модели.
          </p>
          <textarea
            className="w-full p-3 border border-gray-300 rounded-md focus:ring-green-500 focus:border-green-500 text-gray-700"
            rows="4"
            placeholder="Введите текст для токенизации..."
            value={inputText}
            onChange={(e) => setInputText(e.target.value)}
            disabled={isTokenizing || vocab.length === 0}
          ></textarea>
          <button
            onClick={handleTokenizeText}
            className={`mt-4 w-full py-3 px-6 rounded-md text-white font-semibold transition-all duration-300 ease-in-out
              ${isTokenizing || vocab.length === 0 ? 'bg-green-300 cursor-not-allowed' : 'bg-green-600 hover:bg-green-700 focus:outline-none focus:ring-2 focus:ring-green-500 focus:ring-offset-2'}`}
            disabled={isTokenizing || vocab.length === 0}
          >
            {isTokenizing ? 'Токенизация...' : 'Токенизировать текст'}
          </button>
        </div>

        {/* Результаты */}
        <div className="bg-purple-50 p-4 rounded-lg border border-purple-200">
          <h2 className="text-xl font-semibold text-purple-800 mb-3">3. Результаты токенизации</h2>

          <div className="mb-4">
            <h3 className="text-lg font-medium text-gray-700 mb-2">Обученный словарь (имитация):</h3>
            <div className="bg-white p-3 rounded-md border border-gray-200 min-h-[50px] max-h-[200px] overflow-y-auto text-gray-800 text-sm">
              {vocab.length > 0 ? (
                <span className="break-all">{vocab.join(', ')}</span>
              ) : (
                <p className="text-gray-500">Словарь пока пуст. Обучите модель.</p>
              )}
            </div>
          </div>

          <div className="mb-4">
            <h3 className="text-lg font-medium text-gray-700 mb-2">Токены (Pieces):</h3>
            <div className="bg-white p-3 rounded-md border border-gray-200 min-h-[50px] max-h-[200px] overflow-y-auto text-gray-800 text-sm">
              {tokenizedPieces.length > 0 ? (
                <span className="break-all">[{tokenizedPieces.map(p => `'${p}'`).join(', ')}]</span>
              ) : (
                <p className="text-gray-500">Токены пока не сгенерированы.</p>
              )}
            </div>
          </div>

          <div>
            <h3 className="text-lg font-medium text-gray-700 mb-2">ID токенов (IDs):</h3>
            <div className="bg-white p-3 rounded-md border border-gray-200 min-h-[50px] max-h-[200px] overflow-y-auto text-gray-800 text-sm">
              {tokenizedIds.length > 0 ? (
                <span className="break-all">[{tokenizedIds.join(', ')}]</span>
              ) : (
                <p className="text-gray-500">ID токенов пока не сгенерированы.</p>
              )}
            </div>
          </div>

          <button
            onClick={handleClear}
            className="mt-6 w-full py-3 px-6 rounded-md bg-gray-500 text-white font-semibold hover:bg-gray-600 focus:outline-none focus:ring-2 focus:ring-gray-400 focus:ring-offset-2 transition-all duration-300 ease-in-out"
          >
            Очистить результаты
          </button>
        </div>

        <p className="text-center text-gray-500 text-xs mt-8">
          Примечание: Эта демонстрация является упрощенной имитацией работы SentencePiece.
          Фактическая библиотека SentencePiece использует сложные алгоритмы (BPE/Unigram) и реализована на C++ для высокой производительности.
        </p>
      </div>
    </div>
  );
};

export default App;
